# Install latest version

In [1]:
# check latest version
! pip show networkx-neo4j | grep Version

Version: 0.0.3


In [2]:
# ! pip uninstall -y networkx-neo4j #remove the old installation

In [3]:
# ! pip install git+https://github.com/ybaktir/networkx-neo4j #install the latest one

# Add production rules to graph

In [ ]:
from neo4j import GraphDatabase
import nxneo4j as nx
import json
import sys; sys.path.append("..")
from deg.core import Hypergraph, ProductionRuleCorpus, ProductionRule, hg_to_mol

In [5]:
uri = input("Enter your Aura database URI: ")
user = input("Enter your database user name: ")
password = input("Enter your database password: ")

driver = GraphDatabase.driver(uri=uri,auth=(user, password))
                              #OR "bolt://localhost:7673"
                              #OR the cloud url

## LHS

In [6]:
config = {
'node_label': 'Atom_LHS',
'relationship_type': 'BONDED_WITH',
'identifier_property': 'name'
}

In [7]:
G = nx.Graph(driver, config=config)

In [8]:
path = "../examples/grammar.json"
with open(path, 'r') as f:
    raw = json.loads(f.read())

grammar = ProductionRuleCorpus(**raw)

In [9]:
rule = grammar.prod_rule_list[0] # example
lhs = rule.lhs
lhs

Hypergraph(nodes={'bond_28', 'bond_3', 'bond_1'}, edges={'e0'}, node_attributes={'bond_1': {'symbol': BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond'), 'is_in_ring': False, 'visited': False, 'ext_id': 0}, 'bond_3': {'symbol': BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond'), 'is_in_ring': True, 'visited': False, 'ext_id': 1}, 'bond_28': {'symbol': BondSymbol(is_aromatic=False, bond_type=1, stereo=0, type='bond'), 'is_in_ring': True, 'visited': False, 'ext_id': 2}}, edge_attributes={'e0': {'terminal': False, 'symbol': NTSymbol(is_aromatic=False, degree=3, bond_symbol_list=[BondSymbol(is_aromatic=False, bond_type=1, stereo=0, type='bond'), BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond'), BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond')], for_ring=False, symbol='R', type='non_terminal')}}, nodes_in_edge_dict={'e0': ['bond_1', 'bond_3', 'bond_28']}, edge_idx=1)

In [10]:
# add lhs nodes to graph

lhs_atom = [edge for edge in lhs.edge_attributes]

for edge in lhs_atom:
    symbol = lhs.edge_attr(edge)["symbol"].symbol
    type = "NT" if symbol == "R" else "T"
    G.add_node(edge, symbol=symbol, type="atom", atom_type=type)

## RHS

In [11]:
G.node_label = "Atom_RHS"

In [12]:
rhs = rule.rhs
rhs

Hypergraph(nodes={'bond_2', 'bond_28', 'bond_3', 'bond_1'}, edges={'e1', 'e0'}, node_attributes={'bond_2': {'symbol': BondSymbol(is_aromatic=False, bond_type=1, stereo=0, type='bond'), 'is_in_ring': False, 'visited': False}, 'bond_3': {'symbol': BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond'), 'is_in_ring': True, 'visited': False, 'ext_id': 1}, 'bond_28': {'symbol': BondSymbol(is_aromatic=False, bond_type=1, stereo=0, type='bond'), 'is_in_ring': True, 'visited': False, 'ext_id': 2}, 'bond_1': {'symbol': BondSymbol(is_aromatic=False, bond_type=2, stereo=0, type='bond'), 'is_in_ring': False, 'visited': False, 'ext_id': 0}}, edge_attributes={'e0': {'terminal': True, 'is_in_ring': True, 'visited': False, 'NT': False, 'symbol': TSymbol(is_aromatic=False, degree=0, symbol='C', num_explicit_Hs=0, formal_charge=0, chirality=0, type='terminal')}, 'e1': {'terminal': True, 'is_in_ring': False, 'visited': False, 'NT': False, 'symbol': TSymbol(is_aromatic=False, degree=0, symbol='

In [13]:
# add rhs nodes to graph

rhs_atom = [edge for edge in rhs.edge_attributes]

for edge in rhs_atom:
    symbol = rhs.edge_attr(edge)["symbol"].symbol
    type = "NT" if symbol == "R" else "T"
    G.add_node(edge, symbol=symbol, type="atom", atom_type=type)

In [14]:
# make dictionary of atoms and their bonds
# want it to look like 'bond2': 'e0', 'e1'

rhs_edges = rhs.nodes_in_edge_dict
rhs_dict = {}

for edge in rhs_edges:
    for bond in rhs_edges[edge]:
        # if dict has key, append edge to list
        if bond in rhs_dict:
            rhs_dict[bond].append(edge)
        # if dict does not have key, create key and append edge to list
        else:
            rhs_dict[bond] = [edge]

print(rhs_dict)

{'bond_2': ['e0', 'e1'], 'bond_3': ['e0'], 'bond_28': ['e0'], 'bond_1': ['e1']}


In [15]:
# add edges to graph

for bond in rhs_dict:
    if len(rhs_dict[bond]) == 2:
        G.add_edge(rhs_dict[bond][0], rhs_dict[bond][1], bond=bond)

# TO DELETE ALL NODES AND RELATIONSHIPS

In [16]:
G.delete_all()